## Problem Setup

In [1]:
import numpy as np
from scipy.optimize import minimize

# Initialize random seed based on student number
student_number = 1059543
np.random.seed(student_number)

# Define problem parameters
m = 5  # number of lamps
n = 5  # number of surfaces

# Generate random angles theta_k uniformly in [-pi/10, pi/10]
theta_k = np.random.uniform(-np.pi / 10, np.pi / 10, n)

# Generate random distances r_kj uniformly in [0.9, 1.1]
r_kj = np.random.uniform(0.9, 1.1, (n, m))

# Set pmax and Ides with arbitrary values
pmax = 100
Ides = 50

## 3.1

In [2]:
# Define function to compute akj based on the problem definition
def compute_akj(r_kj, theta_k):
    cos_theta = np.maximum(np.cos(theta_k), 0)
    akj = (1 / r_kj**2) * cos_theta[:, np.newaxis]
    return akj

# Compute akj for all surfaces and lamps
akj = compute_akj(r_kj, theta_k)

# Define objective function with a small epsilon to avoid log(0)
epsilon = 1e-10

# Define objective function for the simplified case where pj = p for all lamps
def objective(p):
    Ik = np.dot(akj, np.full(m, p))  # Ik = sum of akj * p
    Ik = np.maximum(Ik, epsilon)     # Prevent Ik from being zero or negative
    return np.max(np.abs(np.log(Ik) - np.log(Ides)))

# Bounds for the power p: [0, pmax]
bounds = [(0, pmax)]

# Initial guess for p
initial_p = [pmax / 2]

# Solve the optimization problem
result = minimize(objective, initial_p, bounds=bounds)

# Output the result
optimal_p = result.x[0]
optimal_cost = result.fun
Ik_optimal = np.dot(akj, np.full(m, optimal_p))
print(f"Optimal power p: {optimal_p}")
print(f"Minimum cost: {optimal_cost}")
print(f"Intensity values Ik: {Ik_optimal}")

Optimal power p: 10.108241457539853
Minimum cost: 0.06504160437078088
Intensity values Ik: [53.36017114 50.41699713 49.96880001 51.51014657 46.8514239 ]


## 3.2

In [3]:
# Define the least squares objective function for the unconstrained problem
def least_squares_objective(p):
    Ik = np.dot(akj, p)  # Ik = sum of akj * pj
    return np.sum((Ik - Ides) ** 2)

# Initial guess for p (for all lamps)
initial_p_unconstrained = np.full(m, pmax / 2)

# Solve the unconstrained least squares problem
result_ls = minimize(least_squares_objective, initial_p_unconstrained)

# Extract the solution
optimal_p_ls = result_ls.x

# Apply the constraints 
optimal_p_ls = np.clip(optimal_p_ls, 0, pmax)  # Ensure 0 <= pj <= pmax
Ik_ls = np.dot(akj, optimal_p_ls)
min_cost_ls = least_squares_objective(optimal_p_ls)

# Prints 
print(f"Optimal power values: {optimal_p_ls}")
print(f"Minimum cost: {min_cost_ls}")
print(f"Intensity values Ik: {Ik_ls}")


Optimal power values: [ 0.48225084  0.         21.780321    5.72216085 21.69659602]
Minimum cost: 0.3916786339973056
Intensity values Ik: [50.28896898 50.32918087 50.27798274 50.26968271 50.22318681]


## 3.3

In [4]:
# Define the weighted least squares objective function with penalty for pj deviations
def weighted_least_squares_objective(p, w):
    Ik = np.dot(akj, p)  # Ik = sum of akj * pj
    penalty = np.sum(w * (p - pmax / 2) ** 2)  # Penalize deviations from pmax/2
    return np.sum((Ik - Ides) ** 2) + penalty

# Initialize weights
w = np.ones(m)

# Define the update function for weights
def update_weights(p, w, alpha=10):
    # Increase the weight where pj violates the bounds
    w_new = w.copy()
    w_new[p < 0] += alpha * np.abs(p[p < 0])  # Increase weight for pj < 0
    w_new[p > pmax] += alpha * (p[p > pmax] - pmax)  # Increase weight for pj > pmax
    return w_new

# Iterative process
tolerance = 1e-3
max_iterations = 100
iteration = 0

while iteration < max_iterations:
    # Solve the weighted least squares problem for the current weights
    result_ls_weighted = minimize(weighted_least_squares_objective, initial_p_unconstrained, args=(w,))
    
    # Get the current solution
    optimal_p_weighted = result_ls_weighted.x
    
    # Check if all pj are within bounds
    if np.all(optimal_p_weighted >= 0) and np.all(optimal_p_weighted <= pmax):
        break  # Solution is feasible, stop the iterations
    
    # Update the weights for the next iteration
    w = update_weights(optimal_p_weighted, w)
    
    iteration += 1


# Compute the intensity values Ik
Ik_weighted = np.dot(akj, optimal_p_weighted)
# Compute the minimum cost using the final solution
min_cost_weighted = weighted_least_squares_objective(optimal_p_weighted, w)
print(f"Optimal power values after weighted least squares: {optimal_p_weighted}")
print(f"Intensity values Ik: {Ik_weighted}")
print(f"Number of iterations: {iteration}")


Optimal power values after weighted least squares: [11.61368366 11.36266886  9.60798787 12.12047823 13.14451174]
Intensity values Ik: [60.47374298 57.22162447 57.83770223 58.57942925 53.96498055]
Number of iterations: 0


## 3.4

In [5]:
from scipy.optimize import linprog

# Prepare linear programming matrices
c = np.zeros(m + 1)  # Objective function coefficients (min t), p1, p2, ..., pm
c[-1] = 1  # We minimize 't'

A_ub = []  # Inequality constraints matrix
b_ub = []  # Inequality constraints vector

# Add constraints for each surface k
for k in range(n):
    # I_k - I_des <= t
    A_ub.append(np.append(akj[k], -1))  # akj * p - t <= Ides
    b_ub.append(Ides)
    
    # I_des - I_k <= t
    A_ub.append(np.append(-akj[k], -1))  # -akj * p - t <= -Ides
    b_ub.append(-Ides)

# Convert to numpy arrays
A_ub = np.array(A_ub)
b_ub = np.array(b_ub)

# Bounds for each pj (0 <= pj <= pmax) and t (no bound on t)
bounds = [(0, pmax)] * m + [(None, None)]  # Bounds for pj and t

# Solve the linear programming problem
result_lp = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Extract the solution
optimal_p_lp = result_lp.x[:-1]  # The p_j values (excluding t)
optimal_t = result_lp.x[-1]      # The optimal t value (max deviation)
# Compute the intensity values Ik
Ik_lp = np.dot(akj, optimal_p_lp)

print(f"Optimal power values (p): {optimal_p_lp}")
print(f"Optimal maximum deviation (t): {optimal_t}")
print(f"Intensity values Ik: {Ik_lp}")


Optimal power values (p): [ 0.70224821  0.         21.54492377  5.48070082 21.68897639]
Optimal maximum deviation (t): 0.02238160626097141
Intensity values Ik: [50.02238161 50.02238161 50.02238161 49.97761839 49.97761839]


## 3.5

In [6]:
# Define the objective function to minimize 't' based on the reformulated problem
def convex_objective(p_and_t):
    # Extract the variables p and t
    p = p_and_t[:-1]
    t = p_and_t[-1]
    
    return t  # We minimize 't'

# Define constraints for the problem
def convex_constraints(p_and_t):
    p = p_and_t[:-1]
    t = p_and_t[-1]
    
    constraints = []
    for k in range(n):
        # Constraint: sum(akj * pj) <= t * Ides
        constraints.append(np.dot(akj[k], p) - t * Ides)
        
        # Constraint: sum(akj * pj) >= Ides / t
        constraints.append(Ides / t - np.dot(akj[k], p))
    
    return np.array(constraints)

# Initial guess for p and t
initial_p_and_t = np.append(np.full(m, pmax / 2), 1.0)  # p starts at pmax/2, t starts at 1

# Define bounds for p and t
bounds_convex = [(0, pmax)] * m + [(1e-6, None)]  # 0 <= p_j <= pmax, t >= 0

# Solve the convex optimization problem
result_convex = minimize(convex_objective, initial_p_and_t, bounds=bounds_convex, constraints={'type': 'ineq', 'fun': convex_constraints})

# Extract the solution
optimal_p_convex = result_convex.x[:-1]  # The p_j values (excluding t)
optimal_t_convex = result_convex.x[-1]   # The optimal t value
# Compute the intensity values Ik
Ik_convex = np.dot(akj, optimal_p_convex)

print(f"Optimal power values (p): {optimal_p_convex}")
print(f"Optimal maximum deviation (t): {optimal_t_convex}")
print(f"Intensity values Ik: {Ik_convex}")

Optimal power values (p): [15.13667617 20.01064388 15.77915926 20.38811373 25.81652136]
Optimal maximum deviation (t): 1.0000000000287557e-06
Intensity values Ik: [100.00008101  96.57569541  98.33764798  97.58583776  91.43685354]
